In [108]:
%load_ext lab_black
%load_ext autoreload

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [109]:
%autoreload 2
# Dataset
from sklearn.datasets import load_diabetes

# Basics
import pandas as pd
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Model
from sklearn.ensemble import RandomForestRegressor

# Helpful:
from sklearn.model_selection import train_test_split

# Path setup
import sys
import os

sys.path.append(r"C:\Users\delon\OneDrive\Python3_new\HIWI\RandomForest_Implementation")
# my functions:
from src.methods.calc_mean import *
from src.methods.calc_weights import *
from src.methods.calc_dist import *

### 1. Load Data

In [92]:
df = pd.read_csv(r"datasets\energy_data_hourly.csv", index_col="datetime")
df.index = pd.to_datetime(df.index)

In [93]:
# Create the 'weekday' column
df["weekday"] = df.index.day_name()
# Create the 'time' column
df["time"] = df.index.time

In [94]:
df["weekday"] = df["weekday"].astype("category")
df = pd.get_dummies(df, columns=["weekday"], prefix="", prefix_sep="")

df["time"] = df["time"].apply(lambda t: t.hour * 60 + t.minute)

### 2. Train Test Split

In [95]:
X_train, X_test, y_train, y_test = train_test_split(
    df.iloc[:, 1:].to_numpy(),
    df["total_energy_usage"].to_numpy(),
    test_size=0.2,
    shuffle=False,
)

### 3. Random Forest

In [99]:
bootstrap = True
max_sample = 0.7

In [104]:
rf = RandomForestRegressor(
    bootstrap=bootstrap, max_samples=max_sample, verbose=2, n_jobs=-1
)

In [105]:
rf.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.2s


building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   11.8s finished


RandomForestRegressor(max_samples=0.7, n_jobs=-1, verbose=2)

### 3. Calculate Weights Random Forest

In [110]:
rf_weights = calc_weights_rf(rf, X_train, X_test, bootstrap, max_sample)

 13%|██████████▏                                                                   | 13/100 [31:46<3:32:35, 146.62s/it]


MemoryError: Unable to allocate 5.54 GiB for an array with shape (13637, 54547) and data type float64

### 4. Calc Mean with weights and normal:

In [10]:
rf_mean_weights = calc_mean_rf(rf_weights, y_train)
rf_mean_normal = rf.predict(X_test)

In [11]:
# Are the two the same?
sum(np.round(rf_mean_weights, 5) == np.round(rf_mean_normal, 5))

89